In [2]:
import sqlite3
import pandas as pd

In [7]:
conn = sqlite3.connect('/home/greenman225/Code/github/donkey_golf/donkey_golf/site.db')
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

,name
0,user
1,pre_tourney
2,rankings
3,teams


In [10]:
pd.read_sql("SELECT * FROM rankings where player like ('%kim%')", conn)

,current_rank,lw_rank,ly_rank,player,events_played
0,55,61.0,61.0,Siwoo Kim,58
1,138,131.0,112.0,Ryuko Tokimatsu,54
2,194,190.0,133.0,Meenwhee Kim,59
3,204,203.0,162.0,Sihwan Kim,49
4,249,246.0,196.0,Kim Koivu,49
5,265,259.0,199.0,Joakim Lagergren,48
6,330,326.0,283.0,Michael Kim,54
7,346,340.0,272.0,K.T. Kim,49
8,352,347.0,288.0,Hyungsung Kim,55
9,355,350.0,287.0,Seunghyuk Kim,55


# Show all tables in our database 

In [21]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

,name
0,user
1,teams
2,rankings
3,pre_tourney
4,leaderboard


In [25]:
pd.read_sql("SELECT * from teams", conn)

,id,tourney_id,golfer
0,4,2748486273828282,Justin Rose
1,4,2748486273828282,Dustin Johnson
2,4,2748486273828282,Rory McIlroy
3,4,2748486273828282,Takumi Kanaya (a)
4,4,2748486273828282,Si Woo Kim
5,4,2748486273828282,Viktor Hovland (a)
6,4,401056527,Justin Rose
7,4,401056527,Dustin Johnson
8,4,401056527,Rory McIlroy
9,4,401056527,Si Woo Kim


In [24]:
pd.read_sql('SELECT * from leaderboard', conn)

,pos,player,to_par,thru,r1,r2,r3,r4,total_strokes,tourney_id
0,T1,Corey Conners,E,1,--,--,--,--,--,401056527
1,T1,Keith Mitchell,E,1,--,--,--,--,--,401056527
2,T1,Kevin Tway,E,1,--,--,--,--,--,401056527
3,T1,Ian Woosnam,E,1,--,--,--,--,--,401056527
4,T5,Andrew Landry,+1,1,--,--,--,--,--,401056527
5,T5,Adam Long,+1,1,--,--,--,--,--,401056527
6,-,Kevin O'Connell (a),E,8:52 AM,--,--,--,--,--,401056527
7,-,Shane Lowry,E,8:52 AM,--,--,--,--,--,401056527
8,-,Mike Weir,E,8:52 AM,--,--,--,--,--,401056527
9,-,Angel Cabrera,E,9:03 AM,--,--,--,--,--,401056527


In [13]:
pd.read_sql('SELECT * from user', conn)

,id,username,email,image_file,password
0,1,first,first@gmail.com,default.jpg,$2b$12$lSacOaPVQw.7JehaPITbSuoUQQHHAzKPrJr9TqH...
1,2,yay,yay@cnn.com,default.jpg,$2b$12$EV2XxOPjCd/GGomJwunxbeaXhib6o4foacxZbAb...
2,3,sumbeech,sumbeech@noneya.com,default.jpg,$2b$12$BRc0.f8qj/.WxwEwhFxOouIIliJjdGFV409tk93...
3,4,poo,poo@poo.com,default.jpg,$2b$12$HLtdo2x2ANCNd96RadWEbektz6QDkqa9c1S1HZi...
4,5,mike,mike@mike.com,default.jpg,$2b$12$mfwZfbokGzgQ3unzRHKVZuswT3xYpPShXb.24PR...
5,6,jeff,jeff@jeff.com,default.jpg,$2b$12$FE3Knp/WrAVizzvitw0jq.3PoElW.iVtRgXuDzT...
